In [ ]:

import numpy as np

import gymnasium as gym  # Correct way to import Gymnasium
from gymnasium.spaces import Dict, Discrete, Box
from minigrid.core.constants import COLOR_NAMES
from minigrid.core.constants import DIR_TO_VEC
from minigrid.core.grid import Grid
from minigrid.core.actions import Actions
from minigrid.core.mission import MissionSpace
from minigrid.core.world_object import Door, Goal, Key, Wall
from minigrid.manual_control import ManualControl
from minigrid.minigrid_env import MiniGridEnv
from gymnasium.utils.play import play
from gymnasium import ObservationWrapper, spaces
from minigrid.wrappers import FullyObsWrapper
import pandas as pd
# Visualization imports
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.special import logsumexp


import os, glob
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
import sys


from moviepy import VideoFileClip, TextClip, CompositeVideoClip
from moviepy import ImageSequenceClip

import random
SEED = 42
np.random.seed(SEED)
import random
random.seed(SEED)

In [ ]:
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = 'Arial'

# workflow for Exploring Museum Features prepped for model training

variable imports

In [ ]:
# pickling varibles
variables_to_save = {
    "museum mask": museum_mask_03m,
    "museum_df": museum_df,
    "state map": state_map,
    
        
}

with open("stored_vars.pkl", "wb") as f:
    pickle.dump(variables_to_save, f)



In [ ]:
demos #import demos from training code 

In [ ]:
# # plot expert features, per time bin. plot trajectories 

        
def plot_expert_features(demos, state_map, museum_df, n_bins=4):
    """
    Plot expert feature expectations (x, y) per time bin.
    """

    W,H = state_map.T.shape

   # same bin logic as before
    bin_edges = np.linspace(0, 1, n_bins + 1)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    # colormap for clusters
    cluster_files = museum_df["file"].tolist()
    n_clusters = len(cluster_files)
    colors = plt.cm.tab20(np.linspace(0, 1, n_clusters))

    # collect and plot per bin
    binned_xy = {b: [] for b in range(n_bins)}

    # iterate through demos (same index as rows of museum_df)
    for cluster_idx, traj in enumerate(demos):
        for (t, x, y, state, visibility) in traj:
            # assign to bin
            bin_idx = np.digitize(t, bin_edges, right=False) - 1
            bin_idx = np.clip(bin_idx, 0, n_bins - 1)
            t_binned = bin_centers[bin_idx]
            binned_xy[bin_idx].append((x, y, cluster_idx))

    # plot per bin
    for b in range(n_bins):
        plt.figure(figsize=(6, 5))
        plt.title(f'Expert Feature Expectations - Bin {b}')
        plt.xlabel('X (normalized)')
        plt.ylabel('Y (normalized)')

        if len(binned_xy[b]) == 0:
            plt.text(0.5, 0.5, 'No data for this bin', ha='center', va='center')
            plt.axis('off')
            plt.show()
            continue

        xs = [p[0] for p in binned_xy[b]]
        ys = [p[1] for p in binned_xy[b]]
        cidx = [p[2] for p in binned_xy[b]]

        for cluster_idx in np.unique(cidx):
            mask = np.array(cidx) == cluster_idx
            plt.scatter(
                np.array(xs)[mask],
                np.array(ys)[mask],
                s=10,
                color=colors[cluster_idx % len(colors)],
                alpha=0.7,
                label=cluster_files[cluster_idx].replace(".csv", "")
            )

        plt.legend(markerscale=2, fontsize=8)
        plt.tight_layout()
        plt.show()

In [ ]:
plot_expert_features(demos, state_map, museum_df, n_bins=4)

In [ ]:
def plot_cluster_features(museum_df, demos, state_map, cluster_idx,  n_bins=4):
    """
    Plot all (x, y) features for a single cluster (trajectory),
    showing points from all time bins together.

    cluster_idx: index of the cluster/row in museum_df.
    """
    W,H = state_map.T.shape
    
    # setup
    cluster_name = museum_df.iloc[cluster_idx]["file"]
    traj = demos[cluster_idx]
    bin_edges = np.linspace(0, 1, n_bins + 1)
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

    xs, ys, colors = [], [], []

    # compute t_binned for each point and store
    for (t, x, y, state, visibility) in traj:
        bin_idx = np.digitize(t, bin_edges, right=False) - 1
        bin_idx = np.clip(bin_idx, 0, n_bins - 1)
        t_binned = bin_centers[bin_idx]
        # store normalized coordinates and bin index for coloring
        xs.append(x * W)
        ys.append(y * H)
        colors.append(bin_idx)  # color by bin

    # plot all bins together, color-coded by bin index
    plt.figure(figsize=(10, 6))
    scatter = plt.scatter(xs, ys, c=colors, cmap="viridis", s=12, alpha=0.8)
    # Overlay the -1 regions from state_map
    obstacles = np.where(np.flipud(state_map) == -1)
    plt.scatter(obstacles[1], obstacles[0], color="lightgray", s=10, alpha=0.6, label="Walls")
    plt.gca().invert_yaxis()
    # plt.colorbar(scatter, label="Time bin")
    cbar = plt.colorbar(scatter, label="Time bin")
    cbar.set_ticks(np.unique(colors)) 
    plt.xlabel("X (normalized)")
    plt.ylabel("Y (normalized)")
    plt.title(f"All Features for {cluster_name}")
    plt.tight_layout()
    plt.show()

In [ ]:
plot_cluster_features(museum_df, demos, state_map,  cluster_idx=5, n_bins=4)